In [35]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import math

In [4]:
# Connectivity Values
DR_API_TOKEN = 'insert_API_Token_service_account_preferred'
DR_MODELING_ENDPOINT = 'http://10.210.20.159' # 'https://app.datarobot.com'
DR_MODELING_HEADERS = {'Content-Type': 'application/json', 'Authorization': 'token %s' % DR_API_TOKEN}

# Minimum and hardcoded worker limits
# Install max will be looked up after this
MIN_WORKERS = 4
MAX_WORKERS = 0 # 0 for unlimited

In [5]:
"""
get maximum license allowed workers
- cluster is assumed to have either this many workers available, or be auto scaled to this max
"""

response = requests.get(
    url = DR_MODELING_ENDPOINT + '/api/v2/clusterLicense/',
    headers=DR_MODELING_HEADERS,
)

if response.status_code != 200:
    print('Request failed; http error {code}: {content}'.format(code=response.status_code, content=response.content))
    exit(1)
else:
    max_workers_total = response.json()['licenseInfo']['concurrentWorkersCount']
    max_workers_allowed = max_workers_total
    if max_workers_allowed > MAX_WORKERS and MAX_WORKERS != 0:
        max_workers_allowed = MAX_WORKERS

In [43]:
"""
get current user limits
"""

response = requests.get(
    url = DR_MODELING_ENDPOINT + '/api/v2/users/?limit=1000',
    headers=DR_MODELING_HEADERS,
)

if response.status_code != 200:
    print('Request failed; http error {code}: {content}'.format(code=response.status_code, content=response.content))
    exit(1)
else:
    df_users = pd.io.json.json_normalize(response.json()['data'])[['id', 'username', 'maxWorkers']]

In [18]:
"""
get current usage
"""

response = requests.get(
    url = DR_MODELING_ENDPOINT + '/api/v2/resourceMonitor/',
    headers=DR_MODELING_HEADERS,
)

if response.status_code != 200:
    print('Request failed; http error {code}: {content}'.format(code=response.status_code, content=response.content))
    exit(1)
else:
    df_activity = json_normalize(response.json()['users'],record_path=['activity'])

In [48]:
num_active_users = len(df_activity)
num_waiting_users = len(df_activity[df_activity['waiting'] > 0])
print('Active users: {active}, Users with jobs in waiting queue: {wait}'.format(active=num_active_users, wait=num_waiting_users))

worker_ceiling = max_workers_allowed

if num_active_users > 1:
    worker_ceiling = math.ceil(max_workers_allowed / num_active_users)

if worker_ceiling < MIN_WORKERS:
    worker_ceiling = MIN_WORKERS

print('Applying global worker ceiling: {ceiling}'.format(ceiling=worker_ceiling))

Active training users: 1, Users with jobs in waiting queue: 0
Applying global worker ceiling: 20


In [47]:
"""
apply ceiling
"""

for user_id in df_users['id']:
    
    print('Applying ceiling to {userid}: {nm}'.format(userid=user_id, nm=df_users[df_users['id'] == user_id]['username'].iloc[0]))
    
    json = {
        "maxWorkers": worker_ceiling
    }

    response = requests.patch(
        url = DR_MODELING_ENDPOINT + '/api/v2/users/' + user_id,
        headers=DR_MODELING_HEADERS,
        json=json
    )
    
    if response.status_code != 204:
        print('Request failed; http error {code}: {content}'.format(code=response.status_code, content=response.content))

Applying ceiling to 6033d34a31288c19413ea26e: admin@datarobot.com
Applying ceiling to 6033e63531288c19413ff01b: mike.taveirne@datarobot.com
Applying ceiling to 6033ed0831288c1941407a73: user1@datarobot.com
Applying ceiling to 6033ed2631288c1941407c6d: user2@datarobot.com
